In [4]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load utils

!pip install TensorboardX
from utils import Logger


from IPython import display


from torch.utils.data import DataLoader
import torchvision
import torch
from torch.optim import Adam
from torch.autograd import Variable

import torch.nn as nn
from torchvision import transforms, datasets
import pandas as pd
from skimage import io, transform
import tensorflow as tf
import logging
from tensorflow import layers

import numpy as np

import os
import zipfile

ValueError: ignored

In [0]:
batch_size = 64

In [6]:
!unzip blues.zip 




unzip:  cannot find or open blues.zip, blues.zip.zip or blues.zip.ZIP.


In [7]:
DATA_FOLDER = 'blues'
dataset = datasets.ImageFolder(DATA_FOLDER,transform=transforms.ToTensor())   # Set dataset

NameError: ignored

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=5, shuffle=True)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img = dataset[0][0]
img = img.numpy()
image = np.squeeze(img[0])
img[0]


plt.imshow(img[0], cmap = 'gray')

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Discriminator(nn.Module):
  
  def __init__(self, input_size, hidden_dim, output_size):
    super(Discriminator, self).__init__()
    
    # Layers
    self.conv1 = nn.Linear(input_size, hidden_dim*4)    # Input Layer
    self.conv2 = nn.Linear(hidden_dim*4, hidden_dim*2)
    self.conv3 = nn.Linear(hidden_dim*2, hidden_dim)
    self.conv4 = nn.Linear(hidden_dim, output_size)  # Final Layer
    
    # Dropout layer
    self.dropout = nn.Dropout(0.3)
    
  def forward(self, x):
    # Flatten image
    x = x.view(-1, 558*544)
      
    x = F.leaky_relu(self.conv1(x), 0.2)   # (input, negative_slope)
    x = self.dropout(x)
    x = F.leaky_relu(self.conv2(x), 0.2)
    x = self.dropout(x)
    x = F.leaky_relu(self.conv3(x), 0.2)
    x = self.dropout(x)
     
    # Final Layer
    out = self.conv4(x)
    return out

In [0]:
class Generator(nn.Module):
  
  def __init__(self, input_size, hidden_dim, output_size):
    super(Generator, self).__init__()
    
    # Layers
    self.conv1 = nn.Linear(input_size, hidden_dim)    # Input Layer
    self.conv2 = nn.Linear(hidden_dim, hidden_dim*2)
    self.conv3 = nn.Linear(hidden_dim*2, hidden_dim*4)
    self.conv4 = nn.Linear(hidden_dim*4, output_size)  # Final Layer
    
    # Dropout layer
    self.dropout = nn.Dropout(0.3)
  
  def forward(self, x):
    x = F.leaky_relu(self.conv1(x), 0.2)   # (input, negative_slope)
    x = self.dropout(x)
    x = F.leaky_relu(self.conv2(x), 0.2)
    x = self.dropout(x)
    x = F.leaky_relu(self.conv3(x), 0.2)
    x = self.dropout(x)
     
    # Final Layer
    output = F.tanh(self.conv4(x))
      
    return output

In [0]:
# Discriminator Hyperarameters

# Size of input image
input_size = 303552
# Size of discriminator output
d_output_size = 1
# Size of last hidden layer in discriminator (layer 3)
d_hidden_size = 32


# Generator Hyperparamters

# Size of latent vector to give to Generator (reffered as noise)
z_size = 100
# Size of discriminator output (generated image)
g_output_size = 303552
# Size of first hidden layer
g_hidden_size = 32

In [0]:
D = Discriminator(input_size, d_hidden_size, d_output_size)
G = Generator(z_size, g_hidden_size, g_output_size)

print(D)
print()
print(G)

In [0]:
# Calculating losses
def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    # label smoothing
    if smooth:
        # smooth, real labels = 0.9
        labels = torch.ones(batch_size)*0.9
    else:
        labels = torch.ones(batch_size) # real labels = 1
        
    # numerically stable loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = torch.zeros(batch_size) # fake labels = 0
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

In [0]:

import torch.optim as optim

# Learning Rate
lr = 0.002

# Optimizers
d_optimizer = optim.Adam(D.parameters(), lr)
g_optimizer = optim.Adam(G.parameters(), lr)

In [0]:
import pickle as pkl

# training hyperparams
num_epochs = 100

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 200

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance
sample_size=5
fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
fixed_z = torch.from_numpy(fixed_z).float()

# train the network
D.train()
G.train()
for epoch in range(num_epochs):
    
    for batch_i, (real_images, _) in enumerate(train_loader):
                
        real_images
        batch_size = real_images.size(0)
        print(batch_size)
        ## Important rescaling step ## 
        real_images = real_images*2 - 5  # rescale input images from [0,1) to [-1, 1)
        print(batch_size)
        
        if(batch_size == 1):
          batch_size=5
        #  TRAIN THE DISCRIMINATOR
        d_optimizer.zero_grad()
        
        # 1. Train with real images

        # Compute the discriminator losses on real images 
        # smooth the real labels
        D_real = D(real_images)
        d_real_loss = real_loss(D_real, smooth=True)
        
        # 2. Train with fake images
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        fake_images = G(z)
        
        # Compute the discriminator losses on fake images        
        D_fake = D(fake_images)
        d_fake_loss = fake_loss(D_fake)
        
        # add up loss and perform backprop
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        

        #  TRAIN THE GENERATOR
        g_optimizer.zero_grad()
        
        # 1. Train with fake images and flipped labels
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        fake_images = G(z)
        
        # Compute the discriminator losses on fake images 
        # using flipped labels!
        D_fake = D(fake_images)
        g_loss = real_loss(D_fake) # use real loss to flip labels
        
        # perform backprop
        g_loss.backward()
        g_optimizer.step()

        # Print some loss stats
        if batch_i % print_every == 0:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))

    
    # AFTER EACH EPOCH
    # append discriminator loss and generator loss
    losses.append((d_loss.item(), g_loss.item()))
    
    # generate and save sample, fake images
    G.eval() # eval mode for generating samples
    samples_z = G(fixed_z)
    samples.append(samples_z)
    G.train() # back to train mode

In [0]:
# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
  pkl.dump(samples, f)

In [0]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator')
plt.plot(losses.T[1], label='Generator')
plt.title("Training Losses")
plt.legend()

In [0]:
# Helper function for viewing a list of passed in sample images
def view_samples(epoch, samples):
    fig, axes = plt.subplots(figsize=(7,7), nrows=4, ncols=4, sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        img = img.detach()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        im = ax.imshow(img, cmap='grey')

In [0]:
# Load samples from generator, taken while training
with open('train_samples.pkl', 'rb') as f:
    samples = pkl.load(f)

In [0]:
rows = 10 # split epochs into 10, so 100/10 = every 10 epochs
cols = 6
fig, axes = plt.subplots(figsize=(7,12), nrows=rows, ncols=cols, sharex=True, sharey=True)

for sample, ax_row in zip(samples[::int(len(samples)/rows)], axes):
    for img, ax in zip(sample[::int(len(sample)/cols)], ax_row):
        img = img.detach()
        ax.imshow(img, cmap='Greys_r')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)